Copyright 2023 Google LLC.

SPDX-License-Identifier: Apache-2.0

# **Text to SQL Generation using PaLM**

In this colab we'll use SQL-PaLM (Codey) to generate SQL statements. 

Reference: [SQL-PALM: IMPROVED LARGE LANGUAGE MODEL ADAPTATION FOR TEXT-TO-SQL](https://arxiv.org/abs/2306.00739)

# **Costs**

In this document, you use the following billable components of Google Cloud:

- BigQuery: You incur costs for the data that you process in BigQuery, including query.
  - Note: We're using BigQuery's public dataset therefore no storage costs will incurred.
- Vertex AI: You incur costs for calls to the Vertex AI service that's represented by the remote model.

# **Workshop Format**

This is a part of hands-on workshop of DevFest 2023

Self Link: https://bit.ly/df23-text-to-sql-generation

For supplement materials, please refer to the [Presentation](https://docs.google.com/presentation/d/1POzklN--NwVaZ44Vqm8nZjey7_8n6NNjrq8i8qwWTes/preview)

### Basic Demo Workflow

- Step-1: Use few-shot prompt methodology to fine tune SQL-PaLM (Codey) for direct SQL generation.
- Step-2: Link SQL-PaLM logic with BigQuery to automate Text-to-Analysis Process.
- Step-3: Create and Store Embeddings for pre-verified SQLs.
- Step-4: Find similar verified Queries

## Environment Preparation

- Library Installation
- Install Vertex AI SDK
- Library Import
- Account Login

### Library Installation

#### Install Vertex AI SDK

In [ ]:
# Only need to run once

!pip install -q google-cloud-aiplatform --upgrade
!pip install -q google-generativeai --upgrade
!pip install -q vertexai --upgrade

: 

#### Install Tabulate

In [ ]:
!pip install -q tabulate --upgrade --user

### Library Import

In [ ]:
import os
import os.path
import errno

import io
import sys
import pandas as pd
import numpy as np

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()
    print("Authenticated")

Authenticated


In [ ]:
import vertexai
from google.cloud import bigquery

PROJECT_ID = "colab-bqml-palm-demo"  # @param {type:"string"}
public_dataset_project = "bigquery-public-data" # do not edit
#public_dataset_id = 'fcc_political_ads' # do not edit
#public_dataset_tables = ['broadcast_tv_radio_station','file_history','file_record','content_info']

public_dataset_id = 'new_york_taxi_trips' # do not edit
public_dataset_tables = ['tlc_yellow_trips_2022','taxi_zone_geom']

vertexai.init(project=PROJECT_ID, location="us-central1")
client = bigquery.Client(project=PROJECT_ID)

In [ ]:
!gcloud services --project $PROJECT_ID \
  enable bigquery.googleapis.com \
  aiplatform.googleapis.com

Operation "operations/acat.p2-436800949134-f1036142-3eec-40ba-b9cd-c7aef0cabd94" finished successfully.


In [ ]:
import re
import pandas as pd
import tabulate
import seaborn as sns
from IPython.display import Markdown, display
from sklearn.metrics.pairwise import cosine_similarity

from google.cloud import aiplatform_v1 as aiplatform

#### Query Test

In [ ]:
def run_select_query(sql_statement):
    """
    Run a BigQuery SQL query and store the results in a table.

    Args:
        sql_statement: The BigQuery SQL query to run.
    """

    query_job = client.query(sql_statement)
    query_results = query_job.result()

    return query_results.to_dataframe()

def print_query_result(query, df):
  print(f'\r\nQuery to execute : {query}')
  print(tabulate.tabulate(df, tablefmt='grid', showindex=False, headers='keys'))

def extract_markdown_block(content):
  # Define the regular expression pattern to match the code block
  pattern = r"```(.+?)```"

  # If you only expect one code block, you can use re.search() to find the first occurrence
  code_block = re.search(pattern, content, re.DOTALL).group(1)
  return code_block

for table_id in public_dataset_tables:
  query = f'select * from `{public_dataset_project}.{public_dataset_id}.{table_id}` limit 3'
  df = run_select_query(query)
  print_query_result(query,df)


Query to execute : select * from `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` limit 3
+-------------+---------------------------+---------------------------+-------------------+-----------------+-------------+----------------------+----------------+---------------+---------+-----------+--------------+----------------+-----------------+---------------+----------------+----------------------+-----------------------+------------------+-------------------+
|   vendor_id | pickup_datetime           | dropoff_datetime          |   passenger_count |   trip_distance |   rate_code | store_and_fwd_flag   |   payment_type |   fare_amount |   extra |   mta_tax |   tip_amount |   tolls_amount |   imp_surcharge |   airport_fee |   total_amount |   pickup_location_id |   dropoff_location_id |   data_file_year |   data_file_month |
+=============+===========================+===========================+===================+=================+=============+======================+=====

In [ ]:
def generate_table_schema():
  pubClient = bigquery.Client(project=public_dataset_project)
  schema_list = []
  for table_id in public_dataset_tables:
    table_ref = pubClient.dataset(public_dataset_id).table(table_id)
    table = pubClient.get_table(table_ref)
    schema_list.append(f'''[SCHEMA details for table `{public_dataset_project}.{public_dataset_id}.{table_id}`]:''')
    for schema in table.schema:
      schema_list.append(
          f'''Full table name: `{public_dataset_project}.{public_dataset_id}.{table_id}` - Column: {schema.name} - Data Type: {schema.field_type} - Primary Key: False - foreign Key: False - Description: {schema.description}'''
      )
  TABLE_SCHEMA_STR = '\n'.join(schema_list).replace("'", "\\'")
  print(TABLE_SCHEMA_STR)

#generate_table_schema()

[SCHEMA details for table `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022`]:
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: vendor_id - Data Type: STRING - Primary Key: False - foreign Key: False - Description: A code indicating the LPEP provider that provided the record. 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc.
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: pickup_datetime - Data Type: TIMESTAMP - Primary Key: False - foreign Key: False - Description: The date and time when the meter was engaged
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: dropoff_datetime - Data Type: TIMESTAMP - Primary Key: False - foreign Key: False - Description: The date and time when the meter was disengaged
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: passenger_count - Data Type: INTEGER - Primary Ke

In [ ]:
TABLE_SCHEMA_STR = f'''
[SCHEMA details for table `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022`]:
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: vendor_id - Data Type: STRING - Primary Key: False - foreign Key: False - Description: A code indicating the LPEP provider that provided the record. 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc.
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: pickup_datetime - Data Type: TIMESTAMP - Primary Key: False - foreign Key: False - Description: The date and time when the meter was engaged
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: dropoff_datetime - Data Type: TIMESTAMP - Primary Key: False - foreign Key: False - Description: The date and time when the meter was disengaged
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: passenger_count - Data Type: INTEGER - Primary Key: False - foreign Key: False - Description: The number of passengers in the vehicle. This is a driver-entered value.
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: trip_distance - Data Type: NUMERIC - Primary Key: False - foreign Key: False - Description: The elapsed trip distance in miles reported by the taximeter.
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: rate_code - Data Type: STRING - Primary Key: False - foreign Key: False - Description: The final rate code in effect at the end of the trip. 1= Standard rate 2=JFK 3=Newark 4=Nassau or Westchester 5=Negotiated fare 6=Group ride
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: store_and_fwd_flag - Data Type: STRING - Primary Key: False - foreign Key: False - Description: This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka \'store and forward,\' because the vehicle did not have a connection to the server. Y= store and forward trip N= not a store and forward trip
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: payment_type - Data Type: STRING - Primary Key: False - foreign Key: False - Description: A numeric code signifying how the passenger paid for the trip. 1= Credit card 2= Cash 3= No charge 4= Dispute 5= Unknown 6= Voided trip
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: fare_amount - Data Type: NUMERIC - Primary Key: False - foreign Key: False - Description: The time-and-distance fare calculated by the meter
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: extra - Data Type: NUMERIC - Primary Key: False - foreign Key: False - Description: Miscellaneous extras and surcharges. Currently, this only includes the $0.50 and $1 rush hour and overnight charges
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: mta_tax - Data Type: NUMERIC - Primary Key: False - foreign Key: False - Description: $0.50 MTA tax that is automatically triggered based on the metered rate in use
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: tip_amount - Data Type: NUMERIC - Primary Key: False - foreign Key: False - Description: Tip amount. This field is automatically populated for credit card tips. Cash tips are not included.
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: tolls_amount - Data Type: NUMERIC - Primary Key: False - foreign Key: False - Description: Total amount of all tolls paid in trip.
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: imp_surcharge - Data Type: NUMERIC - Primary Key: False - foreign Key: False - Description: $0.30 improvement surcharge assessed on hailed trips at the flag drop. The improvement surcharge began being levied in 2015.
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: airport_fee - Data Type: NUMERIC - Primary Key: False - foreign Key: False - Description:
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: total_amount - Data Type: NUMERIC - Primary Key: False - foreign Key: False - Description: The total amount charged to passengers. Does not include cash tips.
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: pickup_location_id - Data Type: STRING - Primary Key: False - foreign Key: True - Description: TLC Taxi Zone in which the taximeter was engaged
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: dropoff_location_id - Data Type: STRING - Primary Key: False - foreign Key: True - Description: TLC Taxi Zone in which the taximeter was disengaged
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: data_file_year - Data Type: INTEGER - Primary Key: False - foreign Key: False - Description: Datafile timestamp year value
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: data_file_month - Data Type: INTEGER - Primary Key: False - foreign Key: False - Description: Datafile timestamp month value
[SCHEMA details for table `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`]:
Full table name: `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` - Column: zone_id - Data Type: STRING - Primary Key: True - foreign Key: False - Description: Unique ID number of each taxi zone. Corresponds with the pickup_location_id and dropoff_location_id in each of the trips tables
Full table name: `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` - Column: zone_name - Data Type: STRING - Primary Key: False - foreign Key: False - Description: Full text name of the taxi zone
Full table name: `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` - Column: borough - Data Type: STRING - Primary Key: False - foreign Key: False - Description: Borough containing the taxi zone
Full table name: `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` - Column: zone_geom - Data Type: GEOGRAPHY - Primary Key: False - foreign Key: False - Description: Geometric outline that defines the taxi zone suitable for GIS analysis.

'''

QUESTION = "[Q]: List top 5 fare of the taxi trips for each borough according to passenger's dropoff location ? Please show the zone name instead of borough id and exclude the trips with zero or negative fare amount." # @param{type:'string'}

## [Table Format] Call SQL-PaLM (Codey) for direct Text-to-SQL generation

### Python Client Initialization (using Model `codey-bison@001`)

*The* Code generation model from PaLM API that you will use in this notebook is `codey-bison@001`.
It is fine-tuned to follow natural language instructions and is suitable for Code Generation and Code Auto Completion

In [ ]:
from vertexai.language_models import TextGenerationModel, \
                                     TextEmbeddingModel, \
                                     CodeGenerationModel, \
                                     InputOutputTextPair

### Use direct code generation (`code-bison@001`)

#### Prompt Preparation

Few-Shot Prompt Methodology will be used here.

Few-shot prompts are a way to teach language models new tasks with just a few examples. Instead of extensive training data, these models can learn from limited examples and perform tasks they haven't seen before. For instance, you can instruct the model with a few samples to do translations, answer questions, or even engage in fictional conversations with specific characters. The model's few-shot learning enables it to adapt quickly to different tasks, making it versatile and useful with minimal training data. It's like giving a quick crash course to the model, allowing it to understand and respond to new scenarios effectively.

In [ ]:
examples = [
    {
      "Question":"[Q]: What are the average fare amoumt, average trip distance, average tip percentage of the total amount and the average fare per mile?",
      "SQL":f'''[SQL]: SELECT ROUND(AVG(tip_amount),2) AS avg_tip,
                  ROUND(AVG(fare_amount),2) AS avg_fare,
                  ROUND(AVG(trip_distance),2) AS avg_distance,
                  ROUND(AVG(tip_proportion),2) AS avg_tip_pct,
                  ROUND(AVG(fare_per_mile),2) AS avg_fare_mile
                FROM (
                  SELECT
                    tip_amount,
                    fare_amount,
                    trip_distance,
                    (tip_amount / fare_amount)*100.0 AS tip_proportion,
                    fare_amount / trip_distance AS fare_per_mile
                  FROM
                    `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022`
                  WHERE
                    trip_distance > 0.01
                    AND fare_amount <100
                    AND payment_type = "1"
                    AND fare_amount > 0 )
      '''
    },{
      "Question":"[Q]: What is the average tip amount when passenger embark or disembark during the midnight (22:00 ~ 03:00) ?",
      "SQL":f'''[SQL]: SELECT ROUND(AVG(tip_amount), 2) avg_tip FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022`
        WHERE payment_type != "6" AND fare_amount > 0 AND (
          (EXTRACT(HOUR FROM pickup_datetime) >= 22 OR EXTRACT(HOUR FROM pickup_datetime) < 3)
          OR
          (EXTRACT(HOUR FROM dropoff_datetime) >= 22 OR EXTRACT(HOUR FROM dropoff_datetime) < 3)
        )
      '''
    },{
       "Question":"[Q]: What are the average fare amoumt, average trip distance, average tip percentage of the total amount and the average fare per mile distinct borogh by passenger\'s pickup location?",
      "SQL":f'''[SQL]: SELECT ROUND(AVG(a.tip_amount),2) AS avg_tip, ROUND(AVG(a.fare_amount),2) AS avg_fare,
                  ROUND(AVG(a.trip_distance),2) AS avg_distance,
                  ROUND(AVG(a.tip_proportion),2) AS avg_tip_pc,
                  ROUND(AVG(a.fare_per_mile),2) AS avg_fare_mile,
                  b.borough AS borough
                FROM ( SELECT
                    pickup_location_id,
                    tip_amount,
                    fare_amount,
                    trip_distance,
                    (tip_amount / fare_amount)*100.0 AS tip_proportion,
                    fare_amount / trip_distance AS fare_per_mile
                  FROM
                    `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022`
                  WHERE
                    trip_distance > 0.01
                    AND fare_amount <100
                    AND payment_type = "1"
                    AND fare_amount > 0 ) AS a
                  JOIN `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` AS b
                  ON a.pickup_location_id = b.zone_id
                GROUP BY b.borough
                ORDER BY avg_fare_mile DESC;
      '''
    }]

def getExamples():
  r = ''
  for example in examples[1:] :
    r = '\r\n'.join([r,'Here is an example of user question and answer SQL.',TABLE_SCHEMA_STR,example['Question'],example['SQL']])
  # print(r)
  return r

def getQuestion():
  return "\r\n".join(['Here is an example of user question and answer SQL.',QUESTION])

In [ ]:
parameters = {
    "temperature": 0.2,
    "max_output_tokens": 1024
}

prefix = '\r\n'.join(['You are a SQL expert. Please convert text into GoogleSQL statement. We will first give the dataset schema and then ask a question in text. You are asked to generate SQL statement.',getExamples(),getQuestion()])

print(prefix)

You are a SQL expert. Please convert text into GoogleSQL statement. We will first give the dataset schema and then ask a question in text. You are asked to generate SQL statement.

Here is an example of user question and answer SQL.

[SCHEMA details for table `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022`]:
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: vendor_id - Data Type: STRING - Primary Key: False - foreign Key: False - Description: A code indicating the LPEP provider that provided the record. 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc.
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: pickup_datetime - Data Type: TIMESTAMP - Primary Key: False - foreign Key: False - Description: The date and time when the meter was engaged
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: dropoff_datetime - Data Type: TIMESTAMP - Primary Ke

#### API Call

In [ ]:
generation_model = CodeGenerationModel.from_pretrained("code-bison@002")

response = generation_model.predict(prefix=prefix,**parameters)

print("Auto Generated SQL is :\r\n")
display(Markdown(response.text))

Auto Generated SQL is :



[SQL]: SELECT zone_name, fare_amount
FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` AS a
JOIN `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` AS b
ON a.zone_id = b.dropoff_location_id
WHERE b.fare_amount > 0
GROUP BY zone_name, fare_amount
ORDER BY fare_amount DESC
LIMIT 5;

Please Note that `Code Direct Generation` method has relatively strict requirement on Input Prompt Length. In this case, we can only give 2 examples. Otherwise, the Model cannot correct interpret the input.

#### Verification

In [ ]:
generated_sql = response.text.replace("[SQL]: ","")
result = run_select_query(generated_sql)
print_query_result(generated_sql,result)


Query to execute : SELECT zone_name, fare_amount
FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` AS a
JOIN `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` AS b
ON a.zone_id = b.dropoff_location_id
WHERE b.fare_amount > 0
GROUP BY zone_name, fare_amount
ORDER BY fare_amount DESC
LIMIT 5;
+-------------------------+---------------+
| zone_name               |   fare_amount |
+=========================+===============+
| Lenox Hill East         |     401092    |
+-------------------------+---------------+
| Lenox Hill East         |     395845    |
+-------------------------+---------------+
| Two Bridges/Seward Park |     187445    |
+-------------------------+---------------+
| LaGuardia Airport       |       9999.99 |
+-------------------------+---------------+
| Park Slope              |       7025    |
+-------------------------+---------------+


### Use Code Chat Generation (`codechat-bison@001`)

#### Promp Preparation

In [ ]:
def getExamplePrompt(index,schema,question,sql):
  prompt = '\r\n'.join([f'Here is the No.{index+1} example. We will give you the data schema, example user query ([Q]), and example GoogleSQL ([SQL]).',
               schema,question,sql])
  print(prompt)
  return prompt

def getFinalPrompt(schema,question):
  prompt = '\r\n'.join(['Now it is your turn to generate GoogleSQL ([SQL]) based on data schema and user query ([Q]). Start.',schema,question])
  print(prompt)
  return prompt

def getGreetingPrompt(exampleNumber):
  prompt = '\r\n'.join([f'You are a SQL expert. We will first give {exampleNumber} examples showing how to generate GoogleSQL using the dataset schema and user query. Please do not answer before it is your turn to answer.'])
  print(prompt)
  return prompt

#### API Call

In [ ]:
from vertexai.language_models import CodeChatModel,CodeChatSession

parameters = {
    "temperature": 0.2,
    "max_output_tokens": 1024
}

chat_model = CodeChatModel.from_pretrained("codechat-bison@001")
chat_model.start_chat()
chat_session = CodeChatSession(chat_model,**parameters)



In [ ]:
exampleNumber = len(examples)
response = chat_session.send_message(getGreetingPrompt(exampleNumber),**parameters)
# print(response.text)

for index,example in enumerate(examples):
  response = chat_session.send_message(getExamplePrompt(index,TABLE_SCHEMA_STR,example["Question"],example["SQL"]),**parameters)
  # print(response.text)

response = chat_session.send_message(getFinalPrompt(TABLE_SCHEMA_STR,QUESTION),**parameters)
display(Markdown(response.text))

You are a SQL expert. We will first give 3 examples showing how to generate GoogleSQL using the dataset schema and user query. Please do not answer before it is your turn to answer.
Here is the No.1 example. We will give you the data schema, example user query ([Q]), and example GoogleSQL ([SQL]).

[SCHEMA details for table `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022`]:
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: vendor_id - Data Type: STRING - Primary Key: False - foreign Key: False - Description: A code indicating the LPEP provider that provided the record. 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc.
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` - Column: pickup_datetime - Data Type: TIMESTAMP - Primary Key: False - foreign Key: False - Description: The date and time when the meter was engaged
Full table name: `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_20

Sure, here is the GoogleSQL generated for this query:

```
SELECT 
  b.borough,
  t.fare_amount
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` AS t
JOIN `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` AS b
ON t.dropoff_location_id = b.zone_id
WHERE t.fare_amount > 0
GROUP BY b.borough, t.fare_amount
ORDER BY t.fare_amount DESC
LIMIT 5;
```

#### Verification

In [ ]:
print(extract_markdown_block(response.text))


SELECT 
  b.borough,
  t.fare_amount
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` AS t
JOIN `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` AS b
ON t.dropoff_location_id = b.zone_id
WHERE t.fare_amount > 0
GROUP BY b.borough, t.fare_amount
ORDER BY t.fare_amount DESC
LIMIT 5;



In [ ]:
# Note: the SQL part will start with a 'sql' which is part of original markdown, so we need to remove it
generated_sql = extract_markdown_block(response.text)
# print(generated_sql)

result = run_select_query(generated_sql)
print_query_result(generated_sql,result)


Query to execute : 
SELECT 
  b.borough,
  t.fare_amount
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` AS t
JOIN `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` AS b
ON t.dropoff_location_id = b.zone_id
WHERE t.fare_amount > 0
GROUP BY b.borough, t.fare_amount
ORDER BY t.fare_amount DESC
LIMIT 5;

+-----------+---------------+
| borough   |   fare_amount |
+===========+===============+
| Manhattan |     401092    |
+-----------+---------------+
| Manhattan |     395845    |
+-----------+---------------+
| Manhattan |     187445    |
+-----------+---------------+
| Queens    |       9999.99 |
+-----------+---------------+
| Brooklyn  |       7025    |
+-----------+---------------+
